In [2]:
!wget https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json

--2024-07-05 12:05:24--  https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/alexeygrigorev/llm-rag-workshop/main/notebooks/documents.json [following]
--2024-07-05 12:05:24--  https://raw.githubusercontent.com/alexeygrigorev/llm-rag-workshop/main/notebooks/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json’

documents.json      100%[===================>] 642.90K  --.-KB/s    in 0.009s  

2024-07-05 12:05:24 (71.9 MB/s) - ‘documents.json’ saved [658332

In [1]:
import json

with open('./documents.json', 'rt') as f_in:
    documents_file = json.load(f_in)

documents = []

for course in documents_file:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [5]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")
es.info()

ObjectApiResponse({'name': '0a746af23672', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'Epx53v-kSz-Dbw94Q7RgEA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [6]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"
response = es.indices.create(index=index_name, body=index_settings)

response

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [9]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es.index(index=index_name, document=doc)

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:31<00:00, 30.43it/s]


In [13]:
def retrieve_documents(query, index_name="course-questions", max_results=5):
    
    search_query = {
        "size": max_results,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es.search(index=index_name, body=search_query)
    documents = [hit['_source'] for hit in response['hits']['hits']]
    return documents

In [14]:
user_question = "How do I join the course after it has started?"

response = retrieve_documents(user_question)

for doc in response:
    print(f"Section: {doc['section']}")
    print(f"Question: {doc['question']}")
    print(f"Answer: {doc['text'][:60]}...\n")

Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to su...

Section: General course-related questions
Question: Course - Can I follow the course after it finishes?
Answer: Yes, we will keep all the materials after the course finishe...

Section: General course-related questions
Question: Course - What can I do before the course starts?
Answer: You can start by installing and setting up all the dependenc...

Section: General course-related questions
Question: How do I use Git / GitHub for this course?
Answer: After you create a GitHub account, you should clone the cour...

Section: Workshop 1 - dlthub
Question: How do I install the necessary dependencies to run the code?
Answer: Answer: To run the provided code, ensure that the 'dlt[duckd...



In [20]:
context_template = """
Section: {section}
Question: {question}
Answer: {text}
""".strip()

prompt_template = """
You're a course teaching assistant.
Answer the user QUESTION based on CONTEXT - the documents retrieved from our FAQ database.
Don't use other information outside of the provided CONTEXT.  

QUESTION: {user_question}

CONTEXT:

{context}
""".strip()


def build_context(documents):
    context_result = ""
    
    for doc in documents:
        doc_str = context_template.format(**doc)
        context_result += ("\n\n" + doc_str)
    
    return context_result.strip()


def build_prompt(user_question, documents):
    context = build_context(documents)
    prompt = prompt_template.format(
        user_question=user_question,
        context=context
    )
    return prompt

def llm(prompt, model="phi3"):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    answer = response.choices[0].message.content
    return answer

def qa_bot(user_question):
    context_docs = retrieve_documents(user_question)
    prompt = build_prompt(user_question, context_docs)
    answer = llm(prompt)
    return answer

In [18]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [22]:
qa_bot("I'm getting invalid reference format: repository name must be lowercase")

" I'm sorry, but I cannot assist with this issue without having direct access to your `profiles.yml` file or specific context from the FAQ database regarding setting environment variables in dbt-postgres jinja templates as they were not provided within CONTEXT itself. Please refer back to Module 2: Workflow Orchestration where it's suggested that updating a line might solve this issue, but without concrete code snippets or error messages from your specific context here on Assistant I cannot accurately provide help for resolving the problem in question format as per original instructions with no other information outside of CONTEXT."

In [23]:
qa_bot("I can't connect to postgres port 5432, my password doesn't work")

' It seems that the issue you\'re encountering is related to password authentication failure for user "root" while trying to connect to PostgreSQL at localhost on port 5432, and possibly also a connection problem with database "ny_taxi". Here are some steps based on the provided context:\n\n1. Ensure that your local installation of postgres is not running before creating a new container using Docker or Terraform since both scenarios could lead to conflicts (e.g., port 5432 issues).\n   \n   If you\'ve already run `docker compose down` and removed the volume, recreate it:\n\n   ```sh\n   docker-compose up -d --build ny_taxi\n   # After this command executes successfully, create a new PostgreSQL service with different port mapping. See point 3 for details on creating services using Terraform/Docker Compose files where you specify the ports to avoid conflicts (e.g., `postgres:15-alpine` Docker image and `-p 5431:5432`).\n   ```\n   \n2. Verify that your local postgres installation is not 

In [24]:
print(_)

 It seems that the issue you're encountering is related to password authentication failure for user "root" while trying to connect to PostgreSQL at localhost on port 5432, and possibly also a connection problem with database "ny_taxi". Here are some steps based on the provided context:

1. Ensure that your local installation of postgres is not running before creating a new container using Docker or Terraform since both scenarios could lead to conflicts (e.g., port 5432 issues).
   
   If you've already run `docker compose down` and removed the volume, recreate it:

   ```sh
   docker-compose up -d --build ny_taxi
   # After this command executes successfully, create a new PostgreSQL service with different port mapping. See point 3 for details on creating services using Terraform/Docker Compose files where you specify the ports to avoid conflicts (e.g., `postgres:15-alpine` Docker image and `-p 5431:5432`).
   ```
   
2. Verify that your local postgres installation is not using port 543

In [25]:
qa_bot("how can I run kafka?")

" To start a Kafka producer using Java in your terminal:\n\n1. Navigate to your project directory where you find build/libs along with src folder containing org/example/JsonProducer.java source code for producing data via kafka topic which is created from this package `org.example`. Here, `<jar_name>` should be replaced by the name of Kafka producer jar file inside 'build' path like: java -cp build/libs/kafka-producer-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n\nFor installing DuckDB In-Memory database with dlt (Data Lifecycle Tool), the context suggests to use `dlt[duckdb]` package which can be installed using pip install command as well as having duckdb itself beforehand: \npip install 'dlt[duckdb]' and ensure that you have DuckDB locally.\n\nLastly, if your Kafka module is not found when running a script named producer.py (producer being an essential part of any data streaming application), it implies there might be issues related to environment variables or d

In [26]:
print(_)

 To start a Kafka producer using Java in your terminal:

1. Navigate to your project directory where you find build/libs along with src folder containing org/example/JsonProducer.java source code for producing data via kafka topic which is created from this package `org.example`. Here, `<jar_name>` should be replaced by the name of Kafka producer jar file inside 'build' path like: java -cp build/libs/kafka-producer-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java

For installing DuckDB In-Memory database with dlt (Data Lifecycle Tool), the context suggests to use `dlt[duckdb]` package which can be installed using pip install command as well as having duckdb itself beforehand: 
pip install 'dlt[duckdb]' and ensure that you have DuckDB locally.

Lastly, if your Kafka module is not found when running a script named producer.py (producer being an essential part of any data streaming application), it implies there might be issues related to environment variables or dependenc

In [27]:
qa_bot("the course has already started, can I still enroll?")

" Based on the context provided, if you have already started taking a course but weren't able to enroll initially or realize it until now, I can provide some general information about continuation options:\n\n- You are still eligible for submission of homeworks as per our FAQs mentioning that even without registration confirmation emails, submissions will be accepted. However, there may have been a deadline passed and you should quickly check the specific course policies or consult with your instructor directly to determine if exceptions can be made given late enrollment circumstances.\n- There's no information regarding retrospective registrations after starting already mentioned in this FAQ section; however, it is possible that individual courses might have their unique guidelines which I recommend checking on the course syllabus or directly contacting your instructor for clarification due to the specific nature of Data Engineering Bootcamp.\n- The materials and support are available

In [28]:
print(_)

 Based on the context provided, if you have already started taking a course but weren't able to enroll initially or realize it until now, I can provide some general information about continuation options:

- You are still eligible for submission of homeworks as per our FAQs mentioning that even without registration confirmation emails, submissions will be accepted. However, there may have been a deadline passed and you should quickly check the specific course policies or consult with your instructor directly to determine if exceptions can be made given late enrollment circumstances.
- There's no information regarding retrospective registrations after starting already mentioned in this FAQ section; however, it is possible that individual courses might have their unique guidelines which I recommend checking on the course syllabus or directly contacting your instructor for clarification due to the specific nature of Data Engineering Bootcamp.
- The materials and support are available thro